In [1]:
import numpy as np

In [2]:
import numpy as np
import pandas as pd

###Struktur des Projekts:###

1.variables.py:
Definiert Spielfeldgrößen, Schiffe und andere Konstanten.

2.clases.py:
Enthält die Klasse Tablero, die das Spielfeld und die Schiffe eines Spielers verwaltet.

3.funciones.py:
Enthält Funktionen wie Schiffsplatzierung, Trefferprüfung und Spielzug-Logik.

4.main.py:
Hauptlogik, die das Spiel steuert und die Spieleraktionen iterativ verarbeitet.

In [5]:
import os
print(os.listdir())


['.git', 'clases.py', 'demo_modus.py', 'funciones.py', 'README.md', 'Schiffe_versenken.main.ipynb', 'variables.py', '__pycache__']


In [6]:
from variables import GRID_SIZE, SHIPS
from clases import Tablero
from funciones import schuss, ist_verloren


In [7]:
#1.variables.py
# Konfiguration des Spiels
GRID_SIZE = 10
SHIPS = [
    (1, 4),  # 4 Schiffe mit Länge 1
    (2, 3),  # 3 Schiffe mit Länge 2
    (3, 2),  # 2 Schiffe mit Länge 3
    (4, 1)   # 1 Schiff mit Länge 4
]


In [8]:
#clases.py
import numpy as np
import random

class Tablero:
    def __init__(self, jugador_id, dimension):
        self.jugador_id = jugador_id
        self.dimension = dimension
        self.tablero = np.full((dimension, dimension), "~")  # "~" für Wasser
        self.impactos = np.full((dimension, dimension), "~")  # "X" für Treffer, "O" für Fehlschuss
        self.barcos = []

    def platzieren_schiffe(self, schiffe):
        for länge, anzahl in schiffe:
            for _ in range(anzahl):
                while True:
                    x = random.randint(0, self.dimension - 1)
                    y = random.randint(0, self.dimension - 1)
                    richtung = random.choice(["H", "V"])  # Horizontal oder Vertikal
                    if self.kann_schiff_platzieren(x, y, länge, richtung):
                        self.setze_schiff(x, y, länge, richtung)
                        break

    def kann_schiff_platzieren(self, x, y, länge, richtung):
        if richtung == "H" and y + länge <= self.dimension:
            return all(self.tablero[x, y + i] == "~" for i in range(länge))
        if richtung == "V" and x + länge <= self.dimension:
            return all(self.tablero[x + i, y] == "~" for i in range(länge))
        return False

    def setze_schiff(self, x, y, länge, richtung):
        if richtung == "H":
            for i in range(länge):
                self.tablero[x, y + i] = "S"
        else:
            for i in range(länge):
                self.tablero[x + i, y] = "S"
        self.barcos.append((x, y, länge, richtung))

    def anzeigen(self, verborgen=False):
        for i in range(self.dimension):
            for j in range(self.dimension):
                if verborgen and self.tablero[i, j] == "S":
                    print("~", end=" ")
                else:
                    print(self.tablero[i, j], end=" ")
            print()


In [9]:
#funciones.py
import numpy as np
def schuss(tablero, x, y):
    if tablero.tablero[x, y] == "S":
        print("Treffer! 🚢")
        tablero.tablero[x, y] = "X"
        tablero.impactos[x, y] = "X"
        return True
    elif tablero.tablero[x, y] == "~":
        print("Daneben! 🌊")
        tablero.tablero[x, y] = "O"
        tablero.impactos[x, y] = "O"
        return False
    else:
        print("Hier wurde bereits geschossen.")
        return False

def ist_verloren(tablero):
    return not np.any(tablero.tablero == "S")


In [10]:
import numpy as np
from variables import GRID_SIZE, SHIPS
from clases import Tablero
from funciones import schuss, ist_verloren
import random

def automatischer_test():
    print("Automatischer Testmodus gestartet...")
    
    # Spieler und Computer-Tabellen erstellen
    spieler_tablero = Tablero("Spieler", GRID_SIZE)
    computer_tablero = Tablero("Computer", GRID_SIZE)

    # Schiffe platzieren
    spieler_tablero.platzieren_schiffe(SHIPS)
    computer_tablero.platzieren_schiffe(SHIPS)

    # Spiel starten
    while True:
        print("\nDein Spielfeld:")
        spieler_tablero.anzeigen()
        
        print("\nComputerfeld (verdeckt):")
        computer_tablero.anzeigen(verborgen=True)

        # Automatische Eingaben (Spielerzug)
        x = random.randint(0, GRID_SIZE - 1)
        y = random.randint(0, GRID_SIZE - 1)
        print(f"\nSpieler schießt auf ({x}, {y})")
        schuss(computer_tablero, x, y)
        
        if ist_verloren(computer_tablero):
            print("Der Spieler hat im Testmodus gewonnen!")
            break

        # Computerzug
        print("\nZug des Computers...")
        while True:
            x, y = random.randint(0, GRID_SIZE - 1), random.randint(0, GRID_SIZE - 1)
            print(f"Computer schießt auf ({x}, {y})")
            if schuss(spieler_tablero, x, y):
                break
        
        if ist_verloren(spieler_tablero):
            print("Der Computer hat im Testmodus gewonnen!")
            break

if __name__ == "__main__":
    automatischer_test()


Automatischer Testmodus gestartet...

Dein Spielfeld:
~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 

Computerfeld (verdeckt):
~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 

Spieler schießt auf (7, 3)
Daneben! 🌊
Der Spieler hat im Testmodus gewonnen!


In [11]:
import numpy as np
import random
import time
from demo_modus import demo_modus



if __name__ == "__main__":
    print("🎮 Willkommen bei Schiffe Versenken!")
    print("Wähle einen Modus:")
    print("1. Spieler gegen Computer")
    print("2. Demoversion (Simulation)")

    wahl = input("Deine Wahl (1/2): ")

    if wahl == "1":
        main()
    elif wahl == "2":
        demo_modus()
    else:
        print("Ungültige Eingabe. Programm wird beendet.")


##4.main.py
from variables import GRID_SIZE, SHIPS
from clases import Tablero
from funciones import schuss, ist_verloren
from colorama import Fore, Style
print(Fore.RED + "Daneben!" + Style.RESET_ALL)
print(Fore.GREEN + "Treffer!" + Style.RESET_ALL)
time.sleep(1)



# def demo_modus():
#     print("🎮 Willkommen im Demomodus von 'Hundir la Flota'!")
#     print("Hier wird das Spiel automatisch simuliert.\n")
    
#     # Spielfelder für Spieler und Computer erstellen
#     spieler_tablero = Tablero("Spieler", GRID_SIZE)
#     computer_tablero = Tablero("Computer", GRID_SIZE)

#     # Schiffe auf beiden Spielfeldern platzieren
#     spieler_tablero.platzieren_schiffe(SHIPS)
#     computer_tablero.platzieren_schiffe(SHIPS)

#     # Spielverlauf automatisch durchführen
#     while True:
#         # Zeige den aktuellen Stand der Spielfelder
#         print("\nDein Spielfeld:")
#         spieler_tablero.anzeigen()

#         print("\nComputerfeld (verdeckt):")
#         computer_tablero.anzeigen(verborgen=True)

#         # Spieler schießt
#         x, y = random.randint(0, GRID_SIZE - 1), random.randint(0, GRID_SIZE - 1)
#         print(f"\nSpieler schießt auf ({x}, {y})...")
#         if schuss(computer_tablero, x, y):
#             print("Treffer!")
#         else:
#             print("Daneben!")

#         # Überprüfung: Hat der Spieler gewonnen?
#         if ist_verloren(computer_tablero):
#             print("\n🎉 Der Spieler hat im Demomodus gewonnen!")
#             break

#         # Computer schießt
#         print("\nComputer ist dran...")
#         x, y = random.randint(0, GRID_SIZE - 1), random.randint(0, GRID_SIZE - 1)
#         print(f"Computer schießt auf ({x}, {y})...")
#         if schuss(spieler_tablero, x, y):
#             print("Computer trifft!")
#         else:
#             print("Computer schießt daneben.")

    
#         # Überprüfung: Hat der Computer gewonnen?
#         if ist_verloren(spieler_tablero):
#             print("\n💻 Der Computer hat im Demomodus gewonnen!")
#             break

#     print("\nDanke fürs Zuschauen! 🎮")


def main():
    print("Willkommen zu Schiffe Versenken!")
    
    # Spieler und Computer-Tabellen erstellen
    spieler_tablero = Tablero("Spieler", GRID_SIZE)
    computer_tablero = Tablero("Computer", GRID_SIZE)

    # Schiffe platzieren
    spieler_tablero.platzieren_schiffe(SHIPS)
    computer_tablero.platzieren_schiffe(SHIPS)

    # Spiel starten
    while True:
        print("\nDein Spielfeld:")
        spieler_tablero.anzeigen()
        
        print("\nComputerfeld (verdeckt):")
        computer_tablero.anzeigen(verborgen=True)

        # Spielerzug
        print("\nDein Zug:")
        while True:
            try:
                x = int(input("Zeile eingeben (0-9): "))
                y = int(input("Spalte eingeben (0-9): "))
                if 0<= x < GRID_SIZE and 0 <= y < GRID_SIZE:
                    break
                else:
                    print("Ungültige Eingabe, die Koordinaten sind ausserhalb des Spielfeldes!")
            except ValueError:
                print("Bitte gültige Zahl eingeben!")

        schuss(computer_tablero, x, y)
        
        if ist_verloren(computer_tablero):
            print("Herzlichen Glückwunsch! Du hast gewonnen!")
            break

        # Computerzug
        print("\nZug des Computers...")
        while True:
            x, y = random.randint(0, GRID_SIZE - 1), random.randint(0, GRID_SIZE - 1)
            if schuss(spieler_tablero, x, y):
                break
        
        if ist_verloren(spieler_tablero):
            print("Der Computer hat gewonnen. Versuch es noch einmal!")
            break

if __name__ == "__main__":
    main()


🎮 Willkommen bei Schiffe Versenken!
Wähle einen Modus:
1. Spieler gegen Computer
2. Demoversion (Simulation)
🎮 Willkommen im Demomodus von 'Hundir la Flota'!
Hier wird das Spiel automatisch simuliert.


Dein Spielfeld:
~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 

Computerfeld (verdeckt):
~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 

Spieler schießt auf (0, 6)...
Daneben! 🌊
Daneben!

🎉 Der Spieler hat im Demomodus gewonnen!

Danke fürs Zuschauen! 🎮
Daneben!
Treffer!
Willkommen zu Schiffe Versenken!

Dein Spielfeld:
~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
~ 